# Using spacy to remove stopwords, add lemma and entity


## Load Dataset
Load dataset created in ../Datapreparation/Extract_street_terms_from_street_names.ipynb

In [1]:
#Load all needed libraries
import pandas as pd
import spacy #Our NLP tools
import de_core_news_md #!python -m spacy download de_core_news_md
import fr_core_news_md #!python -m spacy download fr_core_news_md
from IPython.display import Javascript
from pandarallel import pandarallel
import numpy as np

In [2]:
streetnames = pd.read_csv('streetnames.csv', encoding='UTF-8-SIG', sep=';')
streetnames.columns

Index(['Unnamed: 0', 'STR_ESID', 'STN_LABEL', 'COM_FOSNR', 'COM_NAME',
       'COM_CANTON', 'STR_OFFICIAL', 'STR_EASTING', 'STR_NORTHING',
       'STN_LABEL_NO_BI', 'STN_LABEL_NO_TERMS', 'STR_TERMS', 'STR_PREPS',
       'STN_LABEL_FINAL'],
      dtype='object')

In [3]:
#Remove unused columns
streetnames = streetnames.drop(['Unnamed: 0', 'COM_NAME', 'COM_CANTON','STR_EASTING', 'STR_NORTHING','COM_FOSNR',
                        'STR_OFFICIAL','STR_TERMS', 'STR_PREPS', 'STN_LABEL_NO_BI', 'STN_LABEL_NO_TERMS'], axis=1)

In [4]:
#Drop nan
missing_data = pd.DataFrame(
    streetnames.isnull().sum(),
    columns=['Missing Values'])

print(missing_data)

streetnames = streetnames.dropna()

                 Missing Values
STR_ESID                      0
STN_LABEL                     0
STN_LABEL_FINAL             415


## Using spaCy Deutsch

In [5]:
#Load a German language model to do NLP - the models we use will influence our results a lot
nlp = spacy.load('de_core_news_md') #Change fr/de

In [6]:
stopwords = nlp.Defaults.stop_words

#print(len(stopwords))
#print(stopwords)

### Remove stopwords

In [7]:
streetnames['SPACY_DE'] = streetnames['STN_LABEL_FINAL'].str.split(' ')

streetnames['SPACY_DE'] = streetnames['SPACY_DE'].apply(lambda x: [item for item in x if item not in stopwords])

streetnames['SPACY_DE'] = streetnames['SPACY_DE'].str.join(' ')

#Delete spaces at the beginning and end of the string using function 'strip()'
streetnames['SPACY_DE'] = streetnames['SPACY_DE'].str.strip()

In [8]:
#Create subset since spacy is slow for larger datasets
streetnames100 = streetnames[:100].copy()

#Make ist faster
pandarallel.initialize()
#pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Lemmatization
Assigning the base forms of words. For example, the lemma of “was” is “be”, and the lemma of “rats” is “rat”.

In [9]:
def checkForEntity(num):
    doc = nlp(num)
    #print(len(doc))
    if len(doc)>0:
        return doc[0].lemma_


⚠ Attention: This following code may take a while (>5min..)  
Use `parallel_apply` to run query in parallel from `pandarallel`. This needs more CPU but is much faster (~6min)

In [10]:
import ipywidgets
import datetime
#pandarallel.initialize(progress_bar=True)
starttime = datetime.datetime.now()

temp = streetnames["SPACY_DE"].parallel_apply(checkForEntity)
#temp = streetnames["SPACY_DE"].apply(checkForEntity)

print ("Duration: ", datetime.datetime.now()-starttime)

/opt/conda/lib/python3.10/site-packages/pandarallel/data_types/series.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield data[chunk_]


Duration:  0:05:02.553404


In [11]:
temp2 = temp.rename("SPACY_DE_LEMMA")

In [12]:
streetnames = pd.concat((streetnames,temp2), axis=1)

In [13]:
streetnames.head()

,STR_ESID,STN_LABEL,STN_LABEL_FINAL,SPACY_DE,SPACY_DE_LEMMA
0,10023770,Wiedenweg,Wieden,Wieden,Wieden
1,10179192,Wuhrbärgli,Wuhrbärgli,Wuhrbärgli,Wuhrbärgli
2,10140563,Emanuelenweg,Emanuelen,Emanuelen,Emanuel
3,10069457,Löhrweg,Löhr,Löhr,Löhr
4,10096235,Brunngasse,Brunn,Brunn,Brunn


### Named Entity Recognition (NER)
Labelling named “real-world” objects, like persons, companies or locations.

In [ ]:
#for i in streetnames100.SPACY_DE.items():
for i in streetnames.SPACY_DE.items():
    doc = nlp(i[1])
    #print(i[0])
    for ent in doc.ents:
        #print(f"{ent.text:<20}\t{ent.label_:<3}")
        #streetnames100.loc[i[0], 'SPACY_DE_ENT'] = ent.label_
        streetnames.loc[i[0], 'SPACY_DE_ENT'] = ent.label_

## Using spaCy Français

In [ ]:
#Load a German language model to do NLP - the models we use will influence our results a lot
nlp = spacy.load('fr_core_news_md') #Change fr/de

In [ ]:
stopwords = nlp.Defaults.stop_words

#print(len(stopwords))
#print(stopwords)

### Remove stopwords

In [ ]:
streetnames['SPACY_FR'] = streetnames['STN_LABEL_FINAL'].str.split(' ')

streetnames['SPACY_FR'] = streetnames['SPACY_FR'].apply(lambda x: [item for item in x if item not in stopwords])

streetnames['SPACY_FR'] = streetnames['SPACY_FR'].str.join(' ')

#Delete spaces at the beginning and end of the string using function 'strip()'
streetnames['SPACY_FR'] = streetnames['SPACY_FR'].str.strip()

### Lemmatization
Assigning the base forms of words. For example, the lemma of “was” is “be”, and the lemma of “rats” is “rat”.

In [ ]:
#for i in streetnames100.SPACY_FR.items():
for i in streetnames.SPACY_FR.items():
    doc = nlp(i[1])
    for token in doc:
        #print(f"{token.text:<20}\t{token.lemma_:<20}\t{token.pos_:<6}\t{token.is_stop}")
        #streetnames100.loc[i[0], 'SPACY_DE_LEMMA'] = token.lemma_
        streetnames.loc[i[0], 'SPACY_FR_LEMMA'] = token.lemma_


### Named Entity Recognition (NER)
Labelling named “real-world” objects, like persons, companies or locations.

In [ ]:
#for i in streetnames100.SPACY_FR.items():
for i in streetnames.SPACY_FR.items():
    doc = nlp(i[1])
    #print(i[0])
    for ent in doc.ents:
        #print(f"{ent.text:<20}\t{ent.label_:<3}")
        #streetnames100.loc[i[0], 'SPACY_DE_ENT'] = ent.label_
        streetnames.loc[i[0], 'SPACY_FR_ENT'] = ent.label_

## Check results

In [ ]:
most_common = streetnames.groupby("SPACY_FR_ENT").count()
most_common.STR_ESID

In [ ]:
most_common = streetnames.groupby("SPACY_DE_ENT").count()
most_common.STR_ESID

In [ ]:
streetnames.to_csv('spacy-out.csv', encoding='UTF-8-SIG', sep=';')